### Tables manipulation & counting

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
# import data from python analysis

dat <- read.csv("data/test_data.csv") %>% select(-X)

dat <- dat %>% 
  group_by(poem_id) %>% 
  mutate(poem_id = str_remove_all(poem_id, "\\.txt"),
         line = row_number(),
         line_id = paste0(poem_id, "_", line)) %>% 
  ungroup()

glimpse(dat)

Rows: 436
Columns: 6
$ poem_id     <chr> "P_44", "P_44", "P_44", "P_44", "P_44", "P_44", "P_44", "P…
$ rhyme_words <chr> "одеждой", "поля", "надежда", "земля", "безмятежно", "мечт…
$ rhymes      <dbl> 1, 2, 1, 2, 3, 2, 3, NA, 4, 5, 4, 5, 6, NA, 6, NA, NA, 7, …
$ pos         <chr> "S,жен,неод=твор,ед", "S,сред,неод=(вин,мн|род,ед|им,мн)",…
$ line        <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,…
$ line_id     <chr> "P_44_1", "P_44_2", "P_44_3", "P_44_4", "P_44_5", "P_44_6"…


In [19]:
# clean grammar to only pos tags 

dat <- dat %>% 
    mutate(pos2 = str_replace_all(pos, "^(\\w+)(,|=).*", "\\1"))

unique(dat$pos2)

[1] "S"      "ADV"    "A"      "V"      "PART"   "APRO"   "SPRO"   "ADVPRO"

In [21]:
# read texts (not obligatory but for testing)

filelist <- list.files(path = "data", pattern = ".txt", full.names = T)

corpus <- tibble(poem_id = filelist,
                 text = sapply(filelist, read_file))

corpus <- corpus %>% 
  mutate(poem_id = str_remove_all(poem_id, "data/|\\.txt")) %>% 
  separate_longer_delim(text, delim = "\n") %>% 
  filter(text != "") %>% 
  group_by(poem_id) %>% 
  mutate(line = row_number(), 
         line_id = paste0(poem_id, "_", line)) %>% 
  ungroup()

glimpse(corpus)

Rows: 436
Columns: 4
$ poem_id <chr> "P_40", "P_40", "P_40", "P_40", "P_40", "P_40", "P_40", "P_40"…
$ text    <chr> "Цвела лилея полевая:", "Как яркий снег бела, нежна;", "Красой…
$ line    <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,…
$ line_id <chr> "P_40_1", "P_40_2", "P_40_3", "P_40_4", "P_40_5", "P_40_6", "P…


In [23]:
# join tables with lines and rhyme tagging

full_corpus <- left_join(corpus, dat %>% select(line_id, rhyme_words, rhymes, pos2), by = "line_id")

head(full_corpus,10)

poem_id,text,line,line_id,rhyme_words,rhymes,pos2
<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<chr>
P_40,Цвела лилея полевая:,1,P_40_1,полевая,1,A
P_40,"Как яркий снег бела, нежна;",2,P_40_2,нежна,2,A
P_40,"Красой душистою пленяя,",3,P_40_3,пленяя,1,V
P_40,Цветком любви наречена.,4,P_40_4,наречена,2,V
P_40,Долина ею любовалась;,5,P_40_5,любовалась,3,V
P_40,"Журчал приветно светлый ток,",6,P_40_6,ток,4,S
P_40,"Пчела к цветку не прикасалась,",7,P_40_7,прикасалась,3,V
P_40,Ее лелеял ветерок.,8,P_40_8,ветерок,4,S
P_40,"Но буря вдруг, вдали чернея,",9,P_40_9,чернея,5,V


In [24]:
# create columns with rhyme pairs

# to do: cut rhyme "pairs" detected as same rhyme of 3+ rhyming words
full_corpus <- full_corpus %>% 
  group_by(poem_id, rhymes) %>% 
  mutate(rhyme_pair = ifelse(!is.na(rhymes), # remove NA from collapsing
                             paste0(rhyme_words, collapse = " "), 
                             NA)) %>% 
  mutate(pos_pair = ifelse(!is.na(rhymes),
                          paste0(pos2, collapse = " "),
                          NA)) %>%
  ungroup()

head(full_corpus, 10)

poem_id,text,line,line_id,rhyme_words,rhymes,pos2,rhyme_pair,pos_pair
<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
P_40,Цвела лилея полевая:,1,P_40_1,полевая,1,A,полевая пленяя,A V
P_40,"Как яркий снег бела, нежна;",2,P_40_2,нежна,2,A,нежна наречена,A V
P_40,"Красой душистою пленяя,",3,P_40_3,пленяя,1,V,полевая пленяя,A V
P_40,Цветком любви наречена.,4,P_40_4,наречена,2,V,нежна наречена,A V
P_40,Долина ею любовалась;,5,P_40_5,любовалась,3,V,любовалась прикасалась,V V
P_40,"Журчал приветно светлый ток,",6,P_40_6,ток,4,S,ток ветерок,S S
P_40,"Пчела к цветку не прикасалась,",7,P_40_7,прикасалась,3,V,любовалась прикасалась,V V
P_40,Ее лелеял ветерок.,8,P_40_8,ветерок,4,S,ток ветерок,S S
P_40,"Но буря вдруг, вдали чернея,",9,P_40_9,чернея,5,V,чернея лилея,V S


In [9]:
# count rhyme words freqs

rhyme_words <- full_corpus %>% 
  group_by(rhyme_words) %>% 
  count(sort = T)

head(rhyme_words, 10)

rhyme_words,n
<chr>,<int>
небесах,4
ее,3
любви,3
молодая,3
небесной,3
прах,3
рая,3
балах,2
бестелесной,2


In [12]:
# count rhyme pairs freqs

rhyme_pairs <- full_corpus %>% 
  group_by(rhyme_pair) %>% 
  count(sort = T)

head(rhyme_pairs, 15)

rhyme_pair,n
<chr>,<int>
NA,81
багровый живой слезой рукой тоской,5
пробегает палит сверкает блестит опустошает,5
красу лесу природу свободу,4
подымет прозрит обнимет воскресит,4
колыбели цели были,3
могилы унылый застылый,3
поля земля мечта,3
прах небесах мечтаньях,3


In [25]:
# pos pairs

rhyme_pos <- full_corpus %>% 
    group_by(pos_pair) %>% 
    count(sort = T)

head(rhyme_pos, 15)

pos_pair,n
<chr>,<int>
S S,128
NA,81
V V,58
A A,24
A S,16
V S,14
S V,12
S A,10
S SPRO,8
